In this notebook we are trying to analysis missing points per object

In [2]:
import numpy as np
import os
import json
import skimage
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from matplotlib.patches import Polygon


labels_dir = 'path to labels'
images_dir = 'path to 3dod training'


# Functions


In [3]:
def load_labes(labels_path):
    ann = np.load(labels_path, allow_pickle=True)
    ann = ann.item()
    return ann['masks'], ann['2DBbox'], ann['path']


def get_labels(file):
    with open(file) as f:
        data = json.load(f)
    labels = []
    data = data['data']
    for i in data:
        labels.append(i['label'])
    return labels


def calculate_info(scene_id, masks, depth_conf_path,labels):
    path_raw_arkitscenes = '/mnt/IRONWOLF1/ashkan/ARKitScenes/data/raw/Training/'
    conf_map = np.array(Image.open(os.path.join(
        path_raw_arkitscenes + scene_id + '/confidence', depth_conf_path)))
    depth_map = np.array(Image.open(os.path.join(
        path_raw_arkitscenes + scene_id + '/lowres_depth', depth_conf_path)))
    data = []
    for i in range(len(masks)):
        obj = (masks[i] > 0).astype(np.uint8)
        obj = masks[i]
        # Find points in depth map that are in the object 1 mask
        obj_points = np.where(obj == 1)
        if len(obj_points[0]) == 0:
            data.append({labels[i]:{'percentage': 0, 'depth_size': 0, 'num_points_conf_0_1': 0, 'num_points_obj': 0}})
            continue

        # find number of points in the obj_points with confidence 0 or 1
        conf_0_1 = np.where((conf_map[obj_points[0], obj_points[1]] == 0) | (
            conf_map[obj_points[0], obj_points[1]] == 1))

        percentage = (conf_0_1[0].shape[0] / obj_points[0].shape[0]) * 100
        depth_size = depth_map.shape[0] * depth_map.shape[1]
        num_points_conf_0_1 = conf_0_1[0].shape[0]
        num_points_obj = obj_points[0].shape[0]
        data.append({labels[i]:{'percentage': percentage, 'depth_size': depth_size, 'num_points_conf_0_1': num_points_conf_0_1, 'num_points_obj': num_points_obj}})
    return data


def save_data(data):
    with open('data.json', 'a') as f:
        json.dump(data, f)


# Run the code


In [ ]:
labels = []
images_dir_list = os.listdir(images_dir)

data = []
path_raw_arkitscenes = 'path/ARKitScenes/data/raw/Training/'
for scene_id in os.listdir(labels_dir):
    if len(os.listdir(os.path.join(labels_dir, scene_id, '{}_label'.format(scene_id)))) == 0:
        print("no exist ", scene_id)

    if scene_id in images_dir_list:
        print("Working on > ", scene_id)
        labels = get_labels(os.path.join(images_dir, scene_id, '{}_3dod_annotation.json'.format(scene_id)))

        masks, bboxes, paths = load_labes(os.path.join(labels_dir, scene_id, '{}_label'.format(
            scene_id), '{}_bbox.npy'.format(scene_id)))
        scene_data = []
        for frame in range(len(paths)):
            # Load the depth confidence map
            items = os.listdir(os.path.join(path_raw_arkitscenes, scene_id, 'confidence'))
            try:
                depth_conf_path = items[items.index(paths[frame].split('/')[-1])]
            except:
                continue
            scene_data.append({depth_conf_path: calculate_info(scene_id, masks[frame], depth_conf_path, labels)})
        data.append({scene_id: scene_data})
        print("Done > ", scene_id)
        print("-------------------")

save_data(data)


## Show the results

In [ ]:
# read jsno file as list with dictionary
with open('data.json') as f:
    data = json.load(f)

cnt_all = [0, 0, 0, 0, 0, 0]
cnt_sample =[]
sample_ids = []

# Get all the sample ids
for i in data:
    for key in i.keys():
        sample_ids.append(key)

# create an empty dictionary for cnt_sample
for i in sample_ids:
    cnt_sample.append({i:[0, 0, 0, 0, 0, 0]})


for i in data:
    for key in i.keys():
        for frame in i[key]:
            for key_frame in frame.keys():
                for obj in frame[key_frame]:
                    for key_obj in obj.keys():
                        if obj[key_obj]['percentage'] == 0:
                            cnt_all[0] += 1
                            cnt_sample[sample_ids.index(key)][key][0] += 1
                        if obj[key_obj]['percentage'] > 10:
                            cnt_all[1] += 1
                            cnt_sample[sample_ids.index(key)][key][1] += 1
                        if obj[key_obj]['percentage'] > 20:
                            cnt_all[2] += 1
                            cnt_sample[sample_ids.index(key)][key][2] += 1
                        if obj[key_obj]['percentage'] > 50:
                            cnt_all[3] += 1
                            cnt_sample[sample_ids.index(key)][key][3] += 1
                        if obj[key_obj]['percentage'] > 70:
                            cnt_all[4] += 1
                            cnt_sample[sample_ids.index(key)][key][4] += 1
                        if obj[key_obj]['percentage'] > 90:
                            cnt_all[5] += 1
                            cnt_sample[sample_ids.index(key)][key][5] += 1
                            

In [ ]:
# Draw the bar chart
fig = plt.figure(figsize=(10, 5))
plt.bar(['0%', '10%', '20%', '50%', '70%', '90%'], cnt_all, color='blue', width=0.4)
plt.xlabel('Missing percentage') 
plt.ylabel('Number of objects')
# plt.title('Number of objects with confidence 0 or 1 for all the samples(only objects)')
plt.show()


In [11]:
# print the values for each sample with percentage of points with confidence 0 or 1
print("Number of points with confidence 0 or 1 for all the samples:")
for i in cnt_sample:
    for key in i.keys():
        print("Sample id: ", key)
        print("With missing: ")
        print("0%:{} - 10%:{} - 20%:{} - 50%:{} - 70%:{} - 90%:{} ".format( i[key][0],i[key][1], i[key][2], i[key][3], i[key][4], i[key][5]))
        print("-------------------")
# save the output to a txt file
with open('output.txt', 'w') as f:
    for i in cnt_sample:
        for key in i.keys():
            f.write("Sample id: \n" + key)
            f.write("With missing: \n")
            f.write("0%:{} - 10%:{} - 20%:{} - 50%:{} - 70%:{} - 90%:{} \n".format( i[key][0],i[key][1], i[key][2], i[key][3], i[key][4], i[key][5]))

Number of points with confidence 0 or 1 for all the samples:
Sample id:  40958733
With missing: 
0%:3208 - 10%:442 - 20%:306 - 50%:128 - 70%:84 - 90%:69 
-------------------
Sample id:  41048193
With missing: 
0%:5702 - 10%:627 - 20%:403 - 50%:147 - 70%:92 - 90%:59 
-------------------
Sample id:  41069088
With missing: 
0%:5993 - 10%:283 - 20%:197 - 50%:116 - 70%:84 - 90%:74 
-------------------
Sample id:  41048223
With missing: 
0%:14478 - 10%:1536 - 20%:1246 - 50%:598 - 70%:392 - 90%:243 
-------------------
Sample id:  41048240
With missing: 
0%:13743 - 10%:524 - 20%:306 - 50%:188 - 70%:153 - 90%:131 
-------------------
Sample id:  41007602
With missing: 
0%:7083 - 10%:623 - 20%:348 - 50%:150 - 70%:113 - 90%:91 
-------------------
Sample id:  41069098
With missing: 
0%:1942 - 10%:117 - 20%:52 - 50%:20 - 70%:15 - 90%:11 
-------------------
Sample id:  41048173
With missing: 
0%:34984 - 10%:2943 - 20%:2087 - 50%:954 - 70%:725 - 90%:585 
-------------------
Sample id:  41048246
Wi